<a href="https://colab.research.google.com/github/Sagramine/ScraptKomenIG/blob/main/Scrapt_Komentar_dan_3_Model_Sentiment_Analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SCRAPT KOMENTAR INSTAGRAM**

In [ ]:
# @title Setting DNS

!echo "nameserver 1.1.1.1" | sudo tee /etc/resolv.conf
!echo "nameserver 1.0.0.1" | sudo tee -a /etc/resolv.conf

In [ ]:
# @title Instagram API Wrapper
!pip install instagrapi pandas

In [ ]:
# @title Update API Jika ada
!pip install --upgrade instagrapi

In [ ]:
# @title **Import Libraries dan Modules**
from instagrapi import Client
import pandas as pd
from getpass import getpass
from instagrapi.exceptions import TwoFactorRequired, LoginRequired
from getpass import getpass
import time
import random
import os

In [ ]:
# @title **Login Akun Instagram dan Atur Session**
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

USERNAME = input("Masukkan username Instagram Anda: ")
PASSWORD = getpass("Masukkan password Instagram Anda: ")

# Inisialisasi client
cl = Client()

# Nama file untuk menyimpan sesi di Google Drive
session_file = f"/content/drive/MyDrive/{USERNAME}_session.json"

# Simpan sesi
def save_session(client):
    client.dump_settings(session_file)

# Memuat sesi yang sudah dibuat sebelumnya
def load_session(client):
    if os.path.exists(session_file):
        client.load_settings(session_file)
        return True
    return False

# Percobaan Load sesi yang sudah ada, jika belum login ulang
if load_session(cl):
    try:
        cl.get_timeline_feed()
        print(f"\nBerhasil login menggunakan sesi yang disimpan sebagai: {USERNAME}")
    except Exception as e:
        print("Sesi tidak valid, login ulang...")
        # Jika sesi tidak valid, hapus file sesi dan login ulang
        os.remove(session_file)
        cl = Client()
else:
    print("Tidak ada sesi yang tersimpan, melakukan login...")

# Coba login, jika membutuhkan autentikasi dua faktor, minta kode dari pengguna
try:
    cl.login(USERNAME, PASSWORD)
    save_session(cl)  # Menyimpan sesi setelah login berhasil
    # Verifikasi apakah file sesi disimpan
    if os.path.exists(session_file):
        print(f"\nBerhasil login sebagai: {cl.account_info().username}")
        print(f"File sesi berhasil disimpan di Google Drive: {session_file}")
    else:
        print(f"\nBerhasil login sebagai: {cl.account_info().username}")
        print("Gagal menyimpan file sesi di Google Drive.")
except TwoFactorRequired:
    two_factor_code = input("Masukkan kode autentikasi dua faktor Anda: ")
    cl.login(USERNAME, PASSWORD, verification_code=two_factor_code)
    save_session(cl)  # Menyimpan sesi setelah login berhasil
    # Verifikasi apakah file sesi disimpan
    if os.path.exists(session_file):
        print(f"\nBerhasil login dengan autentikasi dua faktor sebagai: {cl.account_info().username}")
        print(f"File sesi berhasil disimpan di Google Drive: {session_file}")
    else:
        print(f"\nBerhasil login dengan autentikasi dua faktor sebagai: {cl.account_info().username}")
        print("Gagal menyimpan file sesi di Google Drive.")
except Exception as e:
    print(f"Terjadi kesalahan saat login: {e}")

In [ ]:
# @title **Menentukan Fungsi untuk Mengambil Komentar dari Postingan**

def get_comments_and_caption(url, max_comments=110):
    try:
        # Dapatkan media dari URL
        media_pk = cl.media_pk_from_url(url)

        # Ambil informasi media termasuk caption
        media_info = cl.media_info(media_pk)
        caption_text = media_info.caption_text if media_info.caption_text else ""

        # Ambil komentar
        comments = cl.media_comments(media_pk, amount=max_comments)

        # Format data
        comments_data = []
        try:
            for c in comments:
                comments_data.append({
                    'post_caption': caption_text,
                    'username': c.user.username,
                    'profile_url': f"https://instagram.com/{c.user.username}",
                    'comment': c.text,
                    'likes': c.like_count,
                    'timestamp': c.created_at_utc.strftime('%Y-%m-%d %H:%M:%S')
                })
                waktu_tunda = random.uniform(0.5, 1.5)
                print(f"Menunggu selama {waktu_tunda:.2f} detik...")
                time.sleep(waktu_tunda)
            return comments_data

        except Exception as e:
            print(f"Terjadi kesalahan saat memproses komentar: {e.__class__.__name__} - {str(e)}")
            return []

    except Exception as e:
        print(f"Terjadi kesalahan: {e.__class__.__name__} - {str(e)}")
        return []

# Fungsi untuk mendapatkan shortlink dari URL
def get_shortlink(url):
    # Menghapus '/' di awal dan akhir URL jika ada
    url = url.strip('/')
    # Memecah URL menjadi bagian-bagian
    parts = url.split('/')
    # Shortlink biasanya ada di indeks terakhir
    return parts[-1]

In [ ]:
# @title **Atur URL, Jumlah Komentar, dan Hasil**
from google.colab import files

post_urls = []
shortlinks = []

while True:
    url = input("\nMasukkan URL postingan Instagram (atau ketik 'selesai'): ").strip()
    if url.lower() == 'selesai':
        break
    if '/p/' not in url and '/reel/' not in url:
        print("URL tidak valid! Harus mengandung '/p/' atau '/reel/'")
        continue
    post_urls.append(url)
    # Mendapatkan shortlink
    shortlink = get_shortlink(url)
    shortlinks.append(shortlink)

if not post_urls:
    print("Tidak ada URL yang dimasukkan!")
    exit()

# Input jumlah maksimal komentar per post
while True:
    try:
        max_comments = int(input("\nMasukkan jumlah maksimal komentar per post: "))
        if max_comments > 0:
            break
        else:
            print("Jumlah maksimal komentar harus lebih dari 0.")
    except ValueError:
        print("Mohon masukkan angka yang valid.")

# Proses setiap URL
for idx, (url, shortlink) in enumerate(zip(post_urls, shortlinks)):
    print(f"\nMemproses: {url}")
    comments = get_comments_and_caption(url, max_comments)
    print(f"Jumlah komentar yang dikumpulkan: {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Konversi tipe data
        df['likes'] = pd.to_numeric(df['likes'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # Sorting data
        df = df.sort_values(by=['likes', 'timestamp'], ascending=[False, False])

        # Simpan ke CSV dengan nama file menggunakan shortlink
        filename = f"ig_comments_{shortlink}.csv"
        df.to_csv(filename, index=False)

        # Informasi postingan
        try:
            media_pk = cl.media_pk_from_url(url)
            media_info = cl.media_info(media_pk)
            total_post_likes = media_info.like_count
            total_post_comments = media_info.comment_count
            post_link = url  # Link postingan

            # Tambahkan total likes, total comments, dan link postingan di akhir CSV
            with open(filename, 'a', newline='', encoding='utf-8') as f:
                f.write('\n')  # Tambahkan baris kosong
                f.write(f"Total Post Likes,{total_post_likes}\n")
                f.write(f"Total Post Comments,{total_post_comments}\n")
                f.write(f"Post URL,{post_link}\n")

        except Exception as e:
            print(f"Terjadi kesalahan saat mengambil informasi postingan: {e.__class__.__name__} - {str(e)}")

        # Unduh file
        try:
            files.download(filename)
            print(f"\nFile berhasil disimpan dan diunduh: {filename}")
        except Exception as e:
            print(f"Gagal mengunduh file {filename}: {e}")

        print("\nPreview Data:")
        print(df.head())
    else:
        print("\nTidak ada komentar yang berhasil dikumpulkan untuk URL ini.")

    # Cek apakah masih ada URL berikutnya
    if idx < len(post_urls) - 1:
        # Waktu tunda acak antara 3 hingga 5 detik
        waktu_tunda = random.uniform(4, 8)
        print(f"\nMenunggu selama {waktu_tunda:.2f} detik sebelum memproses URL berikutnya...")
        time.sleep(waktu_tunda)
    else:
        print("\nSemua URL telah diproses.")



---



---



---



# **Sentimen Analisis**

In [ ]:
# @title **LIB Indonesia**
!pip install transformers torch sentencepiece
!pip install Sastrawi
!pip install tqdm
!pip install emoji
!pip install openai

In [ ]:
# @title **Pretrain Model Data and Triple Sentiment Analysis Setup**
import re
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import emoji
from tqdm.notebook import tqdm
from openai import OpenAI

# Inisialisasi model pra-latih untuk klasifikasi sentimen Bahasa Indonesia
model_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Konfigurasi API OpenAI.
openai_api_key = "x"
client = OpenAI(api_key=openai_api_key)

# Konfigurasi DeepSeek menggunakan OpenAI SDK
deepseek_api_key = "x"
deepseek_client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

# Fungsi untuk memeriksa apakah karakter merupakan emoji
def is_emoji(character):
    return character in emoji.EMOJI_DATA

# Fungsi preprocessing teks (tanpa stemming)
def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'@[\w\s]+', '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\d+', '', text)
    cleaned_text = ''
    for char in text:
        if char.isalnum() or char.isspace() or is_emoji(char):
            cleaned_text += char
    cleaned_text = ' '.join(cleaned_text.split())
    # Jika hasil pembersihan hanya terdiri dari emoji (tanpa alfanumerik), kembalikan string kosong
    text_no_space = cleaned_text.replace(" ", "")
    if text_no_space and not any(ch.isalnum() for ch in text_no_space):
        return ""
    return cleaned_text

# Gabungkan cleaned_comment dan post_caption sebagai input.
def classify_sentiment_pretrained(cleaned_comment, post_caption):
    combined_text = f"{cleaned_comment} {post_caption}".strip()
    if not combined_text:
        return "Netral"
    inputs = tokenizer(
        combined_text,
        return_tensors='pt',
        max_length=256,
        truncation=True,
        padding='max_length'
    )
    with torch.no_grad():
        outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities).item()
    id2label = {0: "Negatif", 1: "Netral", 2: "Positif"}
    return id2label[predicted_class]

# Fungsi klasifikasi menggunakan API OpenAI.
def classify_sentiment_openai(comment, post_caption):
    if not comment:
        return "Netral"
    prompt = (
        f"Teks komentar: {comment}\n"
        f"Konteks postingan: {post_caption}\n"
        "Berdasarkan informasi di atas, tentukan sentimen komentar tersebut. "
        "Jawab hanya dengan salah satu kata: Positif, Netral, atau Negatif."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Anda adalah asisten analisis sentimen Instagram. Jawaban Anda harus hanya satu kata: Positif, Netral, atau Negatif."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=15,
            temperature=0,
            top_p=1
        )
        raw_response = response.choices[0].message.content.strip()
        for token in raw_response.split():
            token_clean = token.strip().capitalize()
            if token_clean in ["Positif", "Negatif", "Netral"]:
                return token_clean
        return raw_response  # fallback jika tidak ditemukan
    except Exception as e:
        print(f"Error pada OpenAI API: {e.__class__.__name__} - {str(e)}")
        return "Error"

# Fungsi klasifikasi menggunakan DeepSeek
def classify_sentiment_deepseek(comment, post_caption):
    if not comment:
        return "Netral"
    prompt = (
        f"Teks komentar: {comment}\n"
        f"Konteks postingan: {post_caption}\n"
        "Berdasarkan informasi di atas, tentukan sentimen komentar tersebut. "
        "Jawab hanya dengan salah satu kata: Positif, Netral, atau Negatif."
    )
    try:
        response = deepseek_client.chat.completions.create(
            model="deepseek-chat",  # Gunakan "deepseek-chat" atau "deepseek-reasoner"
            messages=[
                {"role": "system", "content": "Anda adalah asisten analisis sentimen Instagram. Jawaban Anda harus hanya satu kata: Positif, Netral, atau Negatif."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=15,
            temperature=0,
            top_p=1,
            stream=False
        )
        raw_response = response.choices[0].message.content.strip()
        for token in raw_response.split():
            token_clean = token.strip().capitalize()
            if token_clean in ["Positif", "Negatif", "Netral"]:
                return token_clean
        return raw_response
    except Exception as e:
        print(f"Error pada DeepSeek API: {e.__class__.__name__} - {str(e)}")
        return "Error"

# Fungsi untuk mengambil kesimpulan dari ketiga metode
def finalize_sentiment(sent_pretrained, sent_openai, sent_deepseek):
    votes = [sent_pretrained, sent_openai, sent_deepseek]
    sentiment_counts = {
        "Positif": votes.count("Positif"),
        "Netral": votes.count("Netral"),
        "Negatif": votes.count("Negatif")
    }
    for sentiment, count in sentiment_counts.items():
        if count >= 2:
            return sentiment
    # Jika ketiga hasil berbeda, gunakan hasil dari OpenAI
    return sent_openai

In [ ]:
# @title **Proses Data dan Analisis Sentimen dengan Tiga Metode**

import glob
import os
from google.colab import drive, files
import pandas as pd
from tqdm.notebook import tqdm

# Mount Google Drive dan buat folder "Sentimen IG" jika belum ada
drive.mount('/content/drive')
output_folder = '/content/drive/My Drive/Sentimen IG'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def process_sentiment_analysis():
    while True:
        print("\nSilakan pilih opsi berikut:")
        print("1. Unggah file CSV")
        print("2. Pilih file CSV yang sudah ada di lingkungan Colab")
        print("3. Batalkan proses")

        choice = input("\nMasukkan pilihan Anda (1/2/3): ").strip()

        if choice == '1':
            print("\nSilakan unggah file CSV yang ingin diproses:")
            uploaded = files.upload()
            if not uploaded:
                print("Tidak ada file yang diunggah! Silakan coba lagi.")
                continue
            selected_files = list(uploaded.keys())
            break
        elif choice == '2':
            csv_files = glob.glob('*.csv')
            if not csv_files:
                print("Tidak ada file CSV di lingkungan kerja. Silakan unggah file CSV terlebih dahulu.")
                continue
            print("\nFile CSV yang tersedia:")
            for idx, file in enumerate(csv_files):
                print(f"{idx+1}. {file}")
            file_choice = input("\nMasukkan nomor file yang ingin diproses (pisahkan dengan koma jika memilih lebih dari satu): ").strip()
            selected_indices = [int(idx.strip()) - 1 for idx in file_choice.split(",") if idx.strip().isdigit()]
            selected_files = [csv_files[idx] for idx in selected_indices if 0 <= idx < len(csv_files)]
            if not selected_files:
                print("Tidak ada file yang dipilih. Silakan coba lagi.")
                continue
            break
        elif choice == '3':
            print("Proses dibatalkan oleh pengguna.")
            return
        else:
            print("Pilihan tidak valid. Silakan masukkan 1, 2, atau 3.")
            continue

    # Memproses file CSV yang dipilih
    all_dfs = []
    output_filenames = []

    for filename in selected_files:
        try:
            df = pd.read_csv(filename)
            print(f"\nKolom yang tersedia dalam file {filename}: {df.columns.tolist()}")
            if 'comment' not in df.columns or 'post_caption' not in df.columns or 'username' not in df.columns:
                print(f"File {filename} tidak memiliki kolom yang diperlukan ('comment', 'post_caption', 'username').")
                continue

            # Hapus duplikasi spam: hapus baris dengan kombinasi username dan comment yang sama
            df = df.drop_duplicates(subset=['username', 'comment'])

            # Buat kolom cleaned_comment dari kolom 'comment' menggunakan fungsi preprocess_text
            df['cleaned_comment'] = df['comment'].apply(preprocess_text)
            # Hapus baris dengan cleaned_comment kosong
            df = df[df['cleaned_comment'].str.strip().ne('')]
            df = df.reset_index(drop=True)
            if df.empty:
                print(f"Tidak ada komentar memenuhi kriteria di {filename}.")
                continue

            print(f"\nMemulai klasifikasi sentimen untuk file {filename}...")
            tqdm.pandas()

            # Klasifikasi menggunakan model pretrained (menggunakan cleaned_comment)
            df['sentimen_pretrained'] = df.progress_apply(
                lambda row: classify_sentiment_pretrained(row['cleaned_comment'], row['post_caption']), axis=1
            )

            # Klasifikasi menggunakan API OpenAI (menggunakan teks asli dari kolom comment)
            df['sentimen_openai'] = df.progress_apply(
                lambda row: classify_sentiment_openai(row['comment'], row['post_caption']), axis=1
            )

            # Klasifikasi menggunakan DeepSeek (menggunakan teks asli dari kolom comment)
            df['sentimen_deepseek'] = df.progress_apply(
                lambda row: classify_sentiment_deepseek(row['comment'], row['post_caption']), axis=1
            )

            # Menyimpulkan hasil dari ketiga metode menggunakan aturan:
            # - Jika dua hasil sama, gunakan hasil tersebut.
            # - Jika ketiganya berbeda, gunakan hasil dari OpenAI.
            df['sentimen_final'] = df.apply(
                lambda row: finalize_sentiment(
                    row['sentimen_pretrained'],
                    row['sentimen_openai'],
                    row['sentimen_deepseek']
                ), axis=1
            )

            # Simpan output ke file CSV di folder "Sentimen IG" di Google Drive; nama file diawali "sentimen_"
            output_filename = os.path.join(output_folder, f"sentimen_{filename}")
            df.to_csv(output_filename, index=False)
            print(f"File hasil disimpan sebagai: {output_filename}")
            all_dfs.append(df)
            output_filenames.append(output_filename)
        except Exception as e:
            print(f"Error processing file {filename}: {str(e)}")
            continue

    # Opsi unduh file hasil individual
    while True:
        download_choice = input("\nApakah Anda ingin mengunduh semua file hasil? (y/n): ").strip().lower()
        if download_choice == 'y':
            for output_filename in output_filenames:
                files.download(output_filename)
                print(f"File {output_filename} sedang diunduh...")
            break
        elif download_choice == 'n':
            print("File hasil tidak diunduh.")
            break
        else:
            print("Pilihan tidak valid. Silakan masukkan 'y' atau 'n'.")

    # Jika lebih dari satu file, tawarkan opsi untuk menggabungkan semua file hasil menjadi satu file
    if len(all_dfs) > 1:
        while True:
            combine_choice = input("\nApakah Anda ingin menggabungkan semua file hasil menjadi satu file? (y/n): ").strip().lower()
            if combine_choice == 'y':
                combined_df = pd.concat(all_dfs, ignore_index=True)
                combined_output_filename = os.path.join(output_folder, "sentimen_combined.csv")
                combined_df.to_csv(combined_output_filename, index=False)
                print(f"\nSemua hasil digabungkan dan disimpan sebagai: {combined_output_filename}")
                while True:
                    combined_download_choice = input(f"Unduh file {combined_output_filename}? (y/n): ").strip().lower()
                    if combined_download_choice == 'y':
                        files.download(combined_output_filename)
                        print(f"File {combined_output_filename} sedang diunduh...")
                        break
                    elif combined_download_choice == 'n':
                        print(f"File {combined_output_filename} tidak diunduh.")
                        break
                    else:
                        print("Pilihan tidak valid. Masukkan 'y' atau 'n'.")
                break
            elif combine_choice == 'n':
                print("File hasil tidak digabungkan.")
                break
            else:
                print("Pilihan tidak valid. Masukkan 'y' atau 'n'.")

    print("\nProses selesai.")

# Jalankan proses analisis sentimen
process_sentiment_analysis()

In [ ]:
# @title **Visualisasi Hasil, Distribusi Sentimen, dan Top 5 Postingan Berdasarkan Sentimen Final**

import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive, files
from tqdm.notebook import tqdm

# Mount Google Drive dan pastikan folder output ada
drive.mount('/content/drive')
output_folder = '/content/drive/My Drive/Sentimen IG'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def visualize_and_generate_top5():
    while True:
        print("\nSilakan pilih opsi berikut:")
        print("1. Unggah file CSV")
        print("2. Pilih file CSV yang sudah ada di lingkungan Colab")
        print("3. Batalkan proses")

        choice = input("\nMasukkan pilihan Anda (1/2/3): ").strip()

        if choice == '1':
            print("\nSilakan unggah file CSV yang ingin divisualisasikan:")
            uploaded = files.upload()
            if not uploaded:
                print("Tidak ada file yang diunggah! Silakan coba lagi.")
                continue
            selected_files = list(uploaded.keys())
            break
        elif choice == '2':
            csv_files = glob.glob('*sentimen*.csv')
            if not csv_files:
                print("\nTidak ada file CSV yang valid di lingkungan kerja. Silakan unggah file terlebih dahulu.")
                continue
            print("\nFile CSV yang tersedia:")
            for idx, file in enumerate(csv_files):
                print(f"{idx+1}. {file}")
            file_choice = input("\nMasukkan nomor file yang ingin divisualisasikan (pisahkan dengan koma jika lebih dari satu): ").strip()
            selected_indices = [int(idx.strip()) - 1 for idx in file_choice.split(",") if idx.strip().isdigit()]
            selected_files = [csv_files[idx] for idx in selected_indices if 0 <= idx < len(csv_files)]
            if not selected_files:
                print("Tidak ada file yang dipilih. Silakan coba lagi.")
                continue
            break
        elif choice == '3':
            print("Proses dibatalkan oleh pengguna.")
            return
        else:
            print("Pilihan tidak valid. Silakan masukkan 1, 2, atau 3.")
            continue

    # Gabungkan data dari semua file CSV yang dipilih
    all_dfs = []
    for filename in selected_files:
        try:
            df = pd.read_csv(filename)
            print(f"\nKolom yang tersedia dalam file {filename}: {df.columns.tolist()}")
            required = ['sentimen_pretrained', 'sentimen_openai', 'sentimen_final']
            if not all(col in df.columns for col in required):
                print(f"File {filename} tidak memiliki kolom yang diperlukan. Lewati file ini.")
                continue
            if 'shortlink' not in df.columns:
                # Jika kolom "shortlink" tidak ada, tambahkan dengan mengekstrak dari nama file
                df['shortlink'] = filename.replace("sentimen_", "").replace(".csv", "")
            all_dfs.append(df)
        except Exception as e:
            print(f"Error membaca file {filename}: {str(e)}")
            continue

    if not all_dfs:
        print("\nTidak ada data yang dapat divisualisasikan.")
        return

    combined_df = pd.concat(all_dfs, ignore_index=True)

    # Visualisasi: Jika ada kolom sentimen_deepseek, tampilkan countplot untuk keempat kolom; jika tidak, tiga kolom.
    if 'sentimen_deepseek' in combined_df.columns:
        fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(24, 6))
        sns.countplot(ax=axes[0], x='sentimen_pretrained', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[0].set_title("Distribusi Sentimen Pretrained")
        sns.countplot(ax=axes[1], x='sentimen_openai', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[1].set_title("Distribusi Sentimen OpenAI")
        sns.countplot(ax=axes[2], x='sentimen_deepseek', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[2].set_title("Distribusi Sentimen DeepSeek")
        sns.countplot(ax=axes[3], x='sentimen_final', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[3].set_title("Distribusi Sentimen Final")
        plt.tight_layout()
        plt.show()
    else:
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 6))
        sns.countplot(ax=axes[0], x='sentimen_pretrained', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[0].set_title("Distribusi Sentimen Pretrained")
        sns.countplot(ax=axes[1], x='sentimen_openai', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[1].set_title("Distribusi Sentimen OpenAI")
        sns.countplot(ax=axes[2], x='sentimen_final', data=combined_df, order=['Positif', 'Netral', 'Negatif'])
        axes[2].set_title("Distribusi Sentimen Final")
        plt.tight_layout()
        plt.show()

    # Menghitung distribusi sentimen untuk masing-masing metode
    total = len(combined_df)
    sentiment_counts_pretrained = combined_df['sentimen_pretrained'].value_counts().reindex(['Positif', 'Netral', 'Negatif'], fill_value=0)
    percentages_pretrained = (sentiment_counts_pretrained / total) * 100

    sentiment_counts_openai = combined_df['sentimen_openai'].value_counts().reindex(['Positif', 'Netral', 'Negatif'], fill_value=0)
    percentages_openai = (sentiment_counts_openai / total) * 100

    sentiment_counts_final = combined_df['sentimen_final'].value_counts().reindex(['Positif', 'Netral', 'Negatif'], fill_value=0)
    percentages_final = (sentiment_counts_final / total) * 100

    if 'sentimen_deepseek' in combined_df.columns:
        sentiment_counts_deepseek = combined_df['sentimen_deepseek'].value_counts().reindex(['Positif', 'Netral', 'Negatif'], fill_value=0)
        percentages_deepseek = (sentiment_counts_deepseek / total) * 100
        print("\nDistribusi Sentimen DeepSeek(%):")
        for sentiment in ['Positif', 'Netral', 'Negatif']:
            print(f"{sentiment}: {sentiment_counts_deepseek[sentiment]} komentar ({percentages_deepseek[sentiment]:.2f}%)")

    print("\nDistribusi Sentimen Pretrained(%):")
    for sentiment in ['Positif', 'Netral', 'Negatif']:
        print(f"{sentiment}: {sentiment_counts_pretrained[sentiment]} komentar ({percentages_pretrained[sentiment]:.2f}%)")

    print("\nDistribusi Sentimen OpenAI(%):")
    for sentiment in ['Positif', 'Netral', 'Negatif']:
        print(f"{sentiment}: {sentiment_counts_openai[sentiment]} komentar ({percentages_openai[sentiment]:.2f}%)")

    print("\nDistribusi Sentimen Final(%):")
    for sentiment in ['Positif', 'Netral', 'Negatif']:
        print(f"{sentiment}: {sentiment_counts_final[sentiment]} komentar ({percentages_final[sentiment]:.2f}%)")

    # Hitung Top 5 Postingan berdasarkan jumlah komentar (berdasarkan sentimen_final)
    grouped = combined_df.groupby("shortlink")["sentimen_final"].value_counts().unstack(fill_value=0)
    if "Positif" not in grouped.columns:
        grouped["Positif"] = 0
    if "Negatif" not in grouped.columns:
        grouped["Negatif"] = 0

    top5_positif = grouped["Positif"].sort_values(ascending=False).head(5)
    top5_negatif = grouped["Negatif"].sort_values(ascending=False).head(5)

    # Tulis hasil top 5 ke dalam file .txt
    top5_filename = "top5_postingan.txt"
    with open(top5_filename, "w", encoding="utf-8") as f:
        f.write("Top 5 Postingan dengan jumlah komentar positif terbanyak\n")
        for i, (shortlink, count) in enumerate(top5_positif.items(), start=1):
            f.write(f"{i}. {shortlink} ({count} komentar positif)\n")
        f.write("\n")
        f.write("Top 5 Postingan dengan jumlah komentar negatif terbanyak\n")
        for i, (shortlink, count) in enumerate(top5_negatif.items(), start=1):
            f.write(f"{i}. {shortlink} ({count} komentar negatif)\n")

    print("\nFile top 5 postingan telah dibuat:", top5_filename)

    download_choice = input("\nUnduh file top5_postingan.txt? (y/n): ").strip().lower()
    if download_choice == 'y':
        files.download(top5_filename)
        print("File top5_postingan.txt sedang diunduh...")
    else:
        print("File top5_postingan.txt tidak diunduh.")

# Jalankan fungsi visualisasi dan pembuatan file top 5
visualize_and_generate_top5()